In [1]:
import numpy as np
import tifffile
from pathlib import Path

def convert_npy_to_mask_tif(input_folder, output_folder):
    """
    Convierte archivos .npy de Cellpose (que contienen máscaras) a archivos .tif
    guardando solo la información de la máscara.

    Args:
        input_folder (str): Ruta a la carpeta que contiene los archivos .npy.
        output_folder (str): Ruta a la carpeta donde se guardarán los archivos .tif.
    """
    input_path = Path(input_folder)
    output_path = Path(output_folder)

    # Crear la carpeta de salida si no existe
    output_path.mkdir(parents=True, exist_ok=True)

    # Listar todos los archivos .npy en la carpeta de entrada
    npy_files = list(input_path.glob('*.npy'))

    if not npy_files:
        print(f"No se encontraron archivos .npy en la carpeta: {input_folder}")
        return

    print(f"Procesando {len(npy_files)} archivo(s) .npy...")

    for npy_file in npy_files:
        try:
            # Cargar el archivo .npy
            # Cellpose guarda un diccionario dentro del npy, por eso allow_pickle=True y .item()
            data = np.load(npy_file, allow_pickle=True).item()

            # Extraer la máscara
            # La clave 'masks' contiene el array NumPy de las máscaras
            mask = data['masks']

            # Asegurarse de que la máscara sea un tipo de dato entero adecuado para TIFF
            # Si la máscara es 0, 1, 2... se puede convertir a uint16 o uint32 si hay muchos objetos
            # Cellpose suele usar int para las etiquetas.
            if mask.dtype != np.uint16 and mask.dtype != np.uint32:
                # Si las etiquetas son pequeñas (ej. < 65535 objetos), uint16 es suficiente.
                # Si son muchas, uint32.
                if mask.max() < 2**16:
                    mask = mask.astype(np.uint16)
                else:
                    mask = mask.astype(np.uint32)

            # Construir el nombre del archivo de salida
            # Ejemplo: "imagen_001_seg.npy" -> "imagen_001_mask.tif"
            base_name = npy_file.stem.replace('_seg', '') # Quita '_seg' si está presente
            output_filename = output_path / f"{base_name}_mask.tif"

            # Guardar la máscara como un archivo TIFF
            # Para imágenes 3D, tifffile guardará automáticamente como un TIFF multipágina.
            tifffile.imwrite(output_filename, mask)

            print(f"Convertido: {npy_file.name} -> {output_filename.name}")

        except KeyError:
            print(f"Advertencia: El archivo {npy_file.name} no contiene la clave 'masks'. Saltando.")
        except Exception as e:
            print(f"Error al procesar {npy_file.name}: {e}")

# --- Configuración de rutas ---
# Asegúrate de que estas rutas sean correctas y accesibles desde tu entorno
input_folder = r"//WS1/WS1_Remote_Disk/Current Segovia Lab/Emilio Gutiérrez/Emilio Gutiérrez SOS/test Cellpose/test set/npy"
output_folder = r"//WS1/WS1_Remote_Disk/Current Segovia Lab/Emilio Gutiérrez/Emilio Gutiérrez SOS/test Cellpose/test set/masks"

if __name__ == "__main__":
    print("Iniciando la conversión de archivos .npy a .tif...")
    convert_npy_to_mask_tif(input_folder, output_folder)
    print("Proceso completado.")

Iniciando la conversión de archivos .npy a .tif...
Procesando 2 archivo(s) .npy...
Convertido: C1-Composite-4_seg.npy -> C1-Composite-4_mask.tif
Convertido: C1-Composite-5_seg.npy -> C1-Composite-5_mask.tif
Proceso completado.
